In [ ]:
%pip install snscrape
%pip install h5py

In [ ]:
import snscrape.modules.twitter as sntwitter
import datetime as dt
from textblob import TextBlob
import json
import numpy as np
from keras.models import Sequential
from keras.layers import GRU,Dense,Embedding
from sklearn.model_selection import train_test_split
import tensorflow as tf

import re
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import pandas as pd

In [ ]:
MEAN_LEN_SENTENCES = 120

In [ ]:
tokenizerJson = open("../input/financialsentimentmodel/tokenizer.json", "r")
tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(tokenizerJson.read())
tokenizerJson.close()
print(list(tokenizer.index_word.values())[:15])
# Tokenized text example

In [ ]:
pattern = "[^a-zA-Z0-9$,.]"
clean = [re.sub(pattern," ","$ESI on lows, down $1.50 to $2.50 BK a real possibility").lower().strip()]
t = tokenizer.texts_to_sequences(clean)
p = pad_sequences(t,maxlen=int(MEAN_LEN_SENTENCES))
print(p)
model = tf.keras.models.load_model("../input/financialsentimentmodel/modelWeights")
print(model.summary())
model.predict(p)

Sentiment polarity: the difference between the number of positive and negative tweets as
a fraction of non-neutral tweets

In [ ]:
ticks = ["AAPL", "AXP", "BA", "CAT", "CSCO", "CVX", "NKE", "GS", "IBM", "INTC", "KO", "MMM", "MSFT", "V", "XOM"]
for T in ticks:  
  tweets = {}
  labels = ['negative', 'neutral', 'positive']
  numdays = 294
  startDate = dt.datetime(2019, 2, 28)
  startDate = dt.datetime(2019, 2, 28)
  rawDates = [(startDate + dt.timedelta(days=x)) for x in range(numdays)]
  date_list = [d.strftime("%Y-%m-%d") for d in rawDates]
  scraped = 0
  print("starting")
  
  for d in range(1, numdays):
  #   i = 0
    polarSum = 0
    currentDate = date_list[d-1]
    s = 0
    freq = {'positive':0, 'negative':0, 'neutral':0}
    if currentDate not in tweets.keys():
      tweets[currentDate] = {'Date': currentDate, 'tweets': [], 'sentiment': 0, 'mode_polar':0, 'neutral': 0, 'volume': 0, 'freq': {}}
    # dailyTweets = []
  #   for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f"(feeling) OR (i am) OR (i'm) OR (i feel) OR (stock) OR (market) OR (economy) OR (trading) since:{date_list[d-1]} until:{date_list[d]} -filter:links -filter:replies").get_items()):
  #   if 
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f"${T} since:{date_list[d-1]} until:{date_list[d]} -filter:links -filter:replies").get_items()):
      if i >= 5000:
        break;
  
      pattern = "[^a-zA-Z0-9$,.]"
      cleaned = [re.sub(pattern," ",tweet.content).lower().strip()]
      tokenized = tokenizer.texts_to_sequences(cleaned)
      padded = pad_sequences(tokenized,maxlen=int(MEAN_LEN_SENTENCES))
      out = model.predict(padded)[0].tolist()
      sentiment = labels[out.index(max(out))]
      freq[sentiment] += 1
  
      entry = {
        'content': tweet.content, 
        'date': currentDate, 
        'user': tweet.username,
        'polarity': sentiment,
        'neuralOuts': list(out)  
      }
  
      tweets[currentDate]['tweets'].append(entry)
      s += 1
  
    lenScraped = len(tweets[currentDate]['tweets']) 
    if lenScraped > 0:
      tweets[currentDate]['mode_polar'] = list(freq.keys())[list(freq.values()).index(max(list(freq.values())))]
    
      scraped += lenScraped
      tweets[currentDate]['volume'] = lenScraped
      tweets[currentDate]['freq'] = freq
      if freq['positive'] + freq['negative'] > 0:
        tweets[currentDate]['sentiment'] = (freq['positive'] - freq['negative'])/(freq['positive'] + freq['negative'])
      else:
        tweets[currentDate]['sentiment'] = 0
    print(f"Ticker: {T}")    
    print(lenScraped)
    print(len(tweets.keys()))
    print(tweets.keys())
  # f = open('tweets.csv', 'w+')
  
  jsonData = json.dumps(tweets)
  f = open(f'tweet{T}RNN.json', 'w+')
  f.write(jsonData)
  f.close()